In [5]:
import scipy.sparse as sparse
import scipy.io as sio
import scipy.stats as stats
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
from time import time
import pandas as pd
# own code
import libarary.graph_tools as gtl
import libarary.invarients as inva
import libarary.cpp as cpp
cpp.test()

hello worlsd
 304278004800


In [6]:
def test(G: nx.Graph):
    print("|V| = ", G.number_of_nodes())
    res = {}
    clocks = []
    start = time()
    #convert
    dists = gtl.convert_dist_dict_to_narray(
        dict(nx.all_pairs_dijkstra_path_length(G)), list(G.nodes)
    )
    end = time()
    clocks.append(end - start)
    print("clock convert: ", clocks[-1])
    #q-extend3
    res["q_ext3_multi"] = cpp.q_extend_multi(dists, 3)
    end = time()
    clocks.append(end - start)
    print("clock q_extend 3 multi: ", clocks[-1], res["q_ext3_multi"] )
    #q-extend4
    start = time()
    res["q_ext4_multi"] = cpp.q_extend_multi(dists, 4)
    end = time()
    clocks.append(end - start)
    print("clock q_extend 4 multi: ", clocks[-1], res["q_ext4_multi"])
    #excess
    start = time()
    res["excess"] = cpp.excess_global_single(dists)
    end = time()
    clocks.append(end - start)
    print("clock excess_global single: ", clocks[-1], res["excess"])
    #q-packing3
    start = time()
    res["q_packing3_multi"] = cpp.q_packing_multi(dists, 3)
    end = time()
    clocks.append(end - start)
    print("clock q_packing 3 ,multi: ", clocks[-1], res["q_packing3_multi"])
    #q-packing4
    start = time()
    res["q_packing4_multi"] = cpp.q_packing_multi(dists, 4)
    end = time()
    clocks.append(end - start)
    print("clock q_packing 4 ,multi: ", clocks[-1], res["q_packing4_multi"])
    return res
res = {}

In [7]:
g_bio_weighted = gtl.read_graph_from_edgelist("networks2/bio-weighted-636/bio-SC-TS.edges")
print("nodes: ", g_bio_weighted.number_of_nodes())
res["bio-weighted"] = test(g_bio_weighted)

nodes:  636
|V| =  636
clock convert:  0.0426487922668457
clock q_extend 3 multi:  1.254302740097046 4.9522111622839596e-06
clock q_extend 4 multi:  808.7499356269836 7.42831674342594e-06
clock excess_global single:  4.492594003677368 19.0
clock q_packing 3 ,multi:  1.2123682498931885 0.25
clock q_packing 4 ,multi:  953.6601314544678 0.25


In [8]:
g_bio_453 = gtl.read_graph_from_edgelist("networks2/bio-celegans-dir/bio-celegans-dir.edges")
print("nodes: ", g_bio_453.number_of_nodes())
res["bio_453"] = test(g_bio_453)

nodes:  453
|V| =  453
clock convert:  0.25521063804626465
clock q_extend 3 multi:  0.7035748958587646 5.86063412061185e-05
clock q_extend 4 multi:  265.25879287719727 9.279337357635429e-05
clock excess_global single:  2.947018623352051 13.0
clock q_packing 3 ,multi:  0.43643760681152344 0.25
clock q_packing 4 ,multi:  264.9243381023407 0.25


In [9]:
mm_ca = sio.mmread("networks2/ca-graph/ca-netscience.mtx")
g_ca : nx.Graph =  nx.from_scipy_sparse_array(mm_ca)

res["ca"] = test(g_ca)

|V| =  379
clock convert:  0.3329899311065674
clock q_extend 3 multi:  0.6121599674224854 0.0001954461057363432
clock q_extend 4 multi:  127.59445714950562 0.0003071295947285393
clock excess_global single:  1.8032777309417725 33.0
clock q_packing 3 ,multi:  0.2673029899597168 0.25
clock q_packing 4 ,multi:  129.4503312110901 0.5


In [10]:
mm_brok = sio.mmread("networks2/brok/brock200-4.mtx")
g_brok : nx.Graph =  nx.from_scipy_sparse_array(mm_ca)
res["brok"] = test(g_brok)

|V| =  379
clock convert:  0.3298006057739258
clock q_extend 3 multi:  0.623042106628418 0.0001954461057363432
clock q_extend 4 multi:  127.71842765808105 0.0003071295947285393
clock excess_global single:  1.7126765251159668 33.0
clock q_packing 3 ,multi:  0.28475260734558105 0.25
clock q_packing 4 ,multi:  136.06236863136292 0.5


In [11]:
pd.DataFrame(res)

,bio-weighted,bio_453,ca,brok
q_ext3_multi,0.000005,0.000059,0.000195,0.000195
q_ext4_multi,0.000007,0.000093,0.000307,0.000307
excess,19.000000,13.000000,33.000000,33.000000
q_packing3_multi,0.250000,0.250000,0.250000,0.250000
q_packing4_multi,0.250000,0.250000,0.500000,0.500000


In [12]:
df = pd.DataFrame(res)
df.to_csv("results/res2.csv")